In [2]:
from ultralytics import YOLO

# --- 1. Load a base model ---
# You can start from scratch or use a pre-trained model for faster convergence.
# 'yolov8n.pt' is small and fast, good for starting.
# You can also use yolov8s.pt, yolov8m.pt, etc., for potentially better accuracy but slower training.
model = YOLO('yolov8n.pt') # Load pre-trained weights

# --- 2. Start Training ---
# Train the model using your 'data.yaml' file
results = model.train(
    data='matriculas.yaml',    # Path to your dataset configuration file
    epochs=50,           # Number of training epochs (adjust as needed, e.g., 50, 100)
    imgsz=640,           # Image size for training (e.g., 640 pixels square)
    batch=16,            # Batch size (adjust based on your GPU memory, e.g., 8, 16, 32)
    name='matriculas_yolov8' # Name for the results folder (e.g., runs/detect/matriculas_yolov8)
    # You can add more arguments here, like learning rate, optimizer, etc.
    # See Ultralytics documentation for all options: https://docs.ultralytics.com/modes/train/
)

# --- 3. (Optional) Evaluate the model after training ---
# results = model.val() # Evaluates performance on the validation set

# --- 4. (Optional) Export the model to a different format ---
# path = model.export(format="onnx") # Example: Export to ONNX format

print("\nTraining complete!")
print("Best model weights saved in: runs/detect/matriculas_yolov8/weights/best.pt") # Path might vary slightly

New https://pypi.org/project/ultralytics/8.3.220 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.215  Python-3.9.23 torch-2.8.0+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=matriculas.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=matriculas_yolov82, nbs=64, nms=False, opset=Non